In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split


In [2]:

# 1. Cargar el dataset
print("Cargando dataset...")
dataset = pd.read_json("../Datasets/dataset_humor_train_embeddings.json", lines=True)

# 2. Extraer y concatenar los embeddings
print("Procesando embeddings...")
we_ft = np.stack(dataset['we_ft'].values) # (N, 300)
we_mx = np.stack(dataset['we_mx'].values) # (N, 300)
we_es = np.stack(dataset['we_es'].values) # (N, 300)


Cargando dataset...
Procesando embeddings...


In [3]:
from torch.utils.data import DataLoader, TensorDataset
def create_minibatches(X, Y, batch_size):
    # Recibe los documentos en X y las etiquetas en Y
    dataset = TensorDataset(X, Y) # Cargar los datos en un dataset de tensores
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    # loader = DataLoader(dataset, batch_size=batch_size)
    return loader

In [4]:
def guardar_resultados(datos, archivo):

    df = pd.DataFrame(datos, columns=['klass'])

    df['id'] = df.index + 1

    df = df[['id', 'klass']]

    df.to_csv(archivo, index=False)

    print(f" {datos} guardado exitosamente! - archivo: {archivo}")

In [15]:

X_full = np.concatenate([we_ft, we_mx, we_es], axis=1) 

Y_full = dataset['klass'].to_numpy()

print(f"Dimensión final de entrada: {X_full.shape}")


Dimensión final de entrada: (10400, 900)


In [18]:

# 3. Dividir en Train y Validation
# Usamos stratify para mantener la proporción de humor/no humor
X_tr, X_val, Y_train, Y_val = train_test_split(
    X_full, Y_full, 
    test_size=0.15, 
    random_state=42, 
    stratify=Y_full
)

print(f"Train: {X_tr.shape}, Val: {X_val.shape}")

Train: (8840, 900), Val: (1560, 900)


In [19]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss(reduction='none')

    def forward(self, inputs, targets):
        ce_loss = self.ce(inputs, targets)
        pt = torch.exp(-ce_loss)
        loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return loss.mean()

In [37]:
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_size, output_size, arquitecture):
        super(MLP, self).__init__()
        
        input_size_h1 = arquitecture[0]
        input_size_h2 = arquitecture[1]
        input_size_h3 = arquitecture[2]

        # Capa 1: Recibe muchas características
        self.fc1 = nn.Linear(input_size, input_size_h1)
        self.bn1 = nn.BatchNorm1d(input_size_h1)
        self.act1 = nn.ReLU()
        self.drop1 = nn.Dropout(0.2)
        
        # Capa 2: Compresión
        self.fc2 = nn.Linear(input_size_h1, input_size_h2)
        self.bn2 = nn.BatchNorm1d(input_size_h2)
        self.act2 = nn.GELU()
        self.drop2 = nn.Dropout(0.5)

        # Capa 3: Refinamiento
        self.fc3 = nn.Linear(input_size_h2, input_size_h3)
        self.bn3 = nn.BatchNorm1d(input_size_h3)
        self.act3 = nn.ReLU()
        self.drop3 = nn.Dropout(0.75)

        # Salida
        self.output = nn.Linear(input_size_h3, output_size)
        
    def forward(self, x):
        x = self.drop1(self.act1(self.bn1(self.fc1(x))))
        x = self.drop2(self.act2(self.bn2(self.fc2(x))))
        x = self.drop3(self.act3(self.bn3(self.fc3(x))))
        x = self.output(x)
        return x

In [27]:
# 1. PREPARAR DATOS

import pandas as pd
import numpy as np

# --- 0. CARGAR EL ARCHIVO DE TEST ---
def test(model):
    print("Cargando dataset de prueba...")
    dataset_test = pd.read_json("../Datasets/dataset_humor_test_embeddings.json", lines=True)

    # --- 1. EXTRACCIÓN Y FUSIÓN DE EMBEDDINGS (CRÍTICO) ---

    print("Procesando vectores...")
    we_ft_test = np.stack(dataset_test['we_ft'].values) # FastText General
    we_mx_test = np.stack(dataset_test['we_mx'].values) # FastText México
    we_es_test = np.stack(dataset_test['we_es'].values) # FastText España


    X_t = np.concatenate([we_ft_test, we_mx_test, we_es_test], axis=1)

    print(f"Dimensiones de Test listas: {X_t.shape}") 


    # Convertir a tensor float
    X_testing = torch.from_numpy(X_t).to(torch.float32)


    device = next(model.parameters()).device 
    X_testing = X_testing.to(device)

    # 2. INFERENCIA
    model.eval() 

    with torch.no_grad(): 
        # Predicción de logits
        y_pred_test_logits = model(X_testing)

    # 3. PROCESAR RESULTADOS
    # Obtener la clase (0 o 1) usando argmax
    y_pred_test_indices = torch.argmax(y_pred_test_logits, dim=1)


    y_pred_final = y_pred_test_indices.cpu().numpy()

    print("Predicciones generadas:")
    print(y_pred_final)


    unique, counts = np.unique(y_pred_final, return_counts=True)
    print("\nConteo de clases predichas:")
    print(dict(zip(unique, counts)))
    return y_pred_final

In [9]:

BATCH_SIZE = 48
LEARNING_RATE = 0.1
EPOCHS = 512 
FACTOR_BALANCE = 1.75

In [28]:
from datetime import datetime

def guardar_log(mensaje, archivo="bitacora_experimentos.txt"):
    ahora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    with open(archivo, "a", encoding="utf-8") as f:
        f.write(f"[{ahora}] {mensaje}\n")

In [ ]:
import torch.optim as optim
import torch
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

def ejecutar_experimento(arquitecture, batch_size, lr, epochs, factor_balance = 1.75, patience = 20):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Preparar Tensores
    X_train_t = torch.from_numpy(X_tr).float()
    Y_train_t = torch.from_numpy(Y_train).long()
    X_val_t = torch.from_numpy(X_val).float()
    Y_val_tensor = torch.from_numpy(Y_val).long().to(device)

    # Parámetros de la red
    input_size = X_tr.shape[1]
    output_size = 2   # 2 clases

    # Inicializar Modelo
    model = MLP(input_size=input_size, output_size=output_size, arquitecture=arquitecture)
    model.to(device)

    classes = np.unique(Y_train)
    weights_calc = compute_class_weight('balanced', classes=classes, y=Y_train)


    weights_calc[1] = weights_calc[1] * factor_balance
    weights = torch.tensor(weights_calc).float().to(device)

    #criterion = nn.CrossEntropyLoss(weight=weights)
    criterion = FocalLoss(alpha=0.5, gamma=2.0) # Prueba alpha entre 0.25 y 1

    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=4)

    # Variables Early Stopping
    best_f1 = 0.0
    epochs_no_improve = 0
    best_model_path = 'mejor_modelo_fusion.pth'

    #print("--- Iniciando Entrenamiento Fusión (900 inputs) ---")

    for epoch in range(epochs + 1):
        model.train()
        dataloader = create_minibatches(X_train_t, Y_train_t, batch_size=batch_size)
        train_loss = 0
        
        for X_b, y_b in dataloader:
            X_b, y_b = X_b.to(device), y_b.to(device)
            
            optimizer.zero_grad()
            output = model(X_b)
            loss = criterion(output, y_b)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            
        # --- VALIDACIÓN ---
        model.eval()
        with torch.no_grad():
            X_v = X_val_t.to(device)
            logits = model(X_v)
            val_loss = criterion(logits, Y_val_tensor).item()
            
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            
            # Métrica Objetivo
            f1_macro = f1_score(Y_val, preds, average='macro')
            acc = accuracy_score(Y_val, preds)
            
            # Métricas por clase para ver si detectamos el humor
            f1_humor = f1_score(Y_val, preds, pos_label=1, average='binary')

        print(f"Ep {epoch+1} | Val Loss: {val_loss:.4f} | F1 Macro: {f1_macro:.4f} (Humor F1: {f1_humor:.4f})")
        
        # Paso del Scheduler
        scheduler.step(val_loss)
        
        # Early Stopping basado en MAXIMIZAR F1 MACRO
        if f1_macro > best_f1:
            best_f1 = f1_macro
            epochs_no_improve = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"¡Nuevo Récord! Modelo guardado.")
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print("Early Stopping activado.")
                break

    # Cargar el mejor y probar
    model.load_state_dict(torch.load(best_model_path))
   

    #post entrenamiento
    X_val_t = torch.from_numpy(X_val).to(torch.float32)


    device = next(model.parameters()).device
    X_val_t = X_val_t.to(device)

    # 2. INFERENCIA
    model.eval() 

    with torch.no_grad(): # Ahorra memoria y cálculo
        # Predicción (Logits)
        y_pred_logits = model(X_val_t)
        
        # Obtener la clase con mayor probabilidad (Argmax)
        y_pred_class_tensor = torch.argmax(y_pred_logits, dim=1)

    # 3. POST-PROCESAMIENTO

    y_pred_val_numpy = y_pred_class_tensor.cpu().numpy()

    #print(y_pred_val_numpy)

    #print("--- Matriz de Confusión ---")
    cm = confusion_matrix(Y_val, y_pred_val_numpy)
    #print(cm)

    print("\n--- Reporte de Clasificación ---")

    print(classification_report(Y_val, y_pred_val_numpy, digits=4, zero_division='warn', target_names=['No Humor', 'Humor']))
    y_pred = test(model)
    nombre_archivo = f"../Resultados_Leo/neuronas{arquitecture[0]}_{arquitecture[1]}_{arquitecture[2]}, {lr}_900_Embeddings_{batch_size}_balance_{factor_balance}.csv"
    print(f"\nExperimento finalizado. Mejor F1 Macro: {best_f1:.4f}")
    return y_pred , nombre_archivo, best_f1
    
    



In [47]:
configuracion = {
    "capas": [512, 256, 512],
    "batch_size": 48,
    "learning_rate": 0.0002,
    "patience": 40,
    "epochs": 256,
    "factor_balance": 0.75
}

y_pred, nombre_archivo, best_f1 = ejecutar_experimento(configuracion["capas"], configuracion["batch_size"], configuracion["learning_rate"],
                                                        configuracion["epochs"], patience=configuracion["patience"], factor_balance=configuracion["factor_balance"])

configuracion['f1_score_best'] = best_f1
texto_a_guardar = f"Experimento #{1}: Config={configuracion}"
   
    
guardar_log(texto_a_guardar, archivo="bitacora_experimentos_exp_29_11_V2.txt")

Ep 1 | Val Loss: 0.0285 | F1 Macro: 0.7665 (Humor F1: 0.7188)
¡Nuevo Récord! Modelo guardado.
Ep 2 | Val Loss: 0.0289 | F1 Macro: 0.7760 (Humor F1: 0.7374)
¡Nuevo Récord! Modelo guardado.
Ep 3 | Val Loss: 0.0271 | F1 Macro: 0.7939 (Humor F1: 0.7502)
¡Nuevo Récord! Modelo guardado.
Ep 4 | Val Loss: 0.0269 | F1 Macro: 0.8069 (Humor F1: 0.7649)
¡Nuevo Récord! Modelo guardado.
Ep 5 | Val Loss: 0.0274 | F1 Macro: 0.7986 (Humor F1: 0.7529)
Ep 6 | Val Loss: 0.0277 | F1 Macro: 0.8074 (Humor F1: 0.7598)
¡Nuevo Récord! Modelo guardado.
Ep 7 | Val Loss: 0.0299 | F1 Macro: 0.7648 (Humor F1: 0.7303)
Ep 8 | Val Loss: 0.0313 | F1 Macro: 0.7792 (Humor F1: 0.7447)
Ep 9 | Val Loss: 0.0307 | F1 Macro: 0.7836 (Humor F1: 0.7451)
Ep 10 | Val Loss: 0.0314 | F1 Macro: 0.7942 (Humor F1: 0.7542)
Ep 11 | Val Loss: 0.0320 | F1 Macro: 0.7943 (Humor F1: 0.7546)
Ep 12 | Val Loss: 0.0330 | F1 Macro: 0.7880 (Humor F1: 0.7500)
Ep 13 | Val Loss: 0.0352 | F1 Macro: 0.7973 (Humor F1: 0.7545)
Ep 14 | Val Loss: 0.0356 | F1 

C:\Users\leopa\AppData\Local\Temp\ipykernel_3456\2076789777.py:94: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


Procesando vectores...
Dimensiones de Test listas: (5600, 900)
Predicciones generadas:
[1 0 0 ... 1 0 1]

Conteo de clases predichas:
{np.int64(0): np.int64(3301), np.int64(1): np.int64(2299)}

Experimento finalizado. Mejor F1 Macro: 0.8074


In [43]:
guardar_resultados(y_pred, "../Resultados_Leo/Prob_drop_29_2.csv")

 [0 1 0 ... 1 0 1] guardado exitosamente! - archivo: ../Resultados_Leo/Prob_drop_29_2.csv


In [16]:
import itertools
import pandas as pd

#capa1_prop = [1024]
#capa2_prop = [1024,512, 256]
#capa3_prop = [1024,512, 256]

batch_size_prop = [48, 56, 64]

lr_prop = [0.01, 0.025, 0.0025, 0.1, 0.25]

factor_balance = [0.5, 0.75, 0.8, 1, 1.25, 1.5, 1.75, 1.85]
resultados = []
experimento = 1
for batch_size, lr, factor in itertools.product(batch_size_prop, lr_prop, factor_balance):

    configuracion = {
        "capas": [256, 128, 64],
        "batch_size": batch_size,
        "learning_rate": lr,
        "patience": 40,
        "epochs": 256,
        "factor_balance": factor
    }

    y_pred, nombre_archivo, best_f1 = ejecutar_experimento(configuracion["capas"], configuracion["batch_size"], configuracion["learning_rate"],
                                                        configuracion["epochs"], patience=configuracion["patience"], factor_balance=configuracion["factor_balance"])

    configuracion['f1_score_best'] = best_f1
    texto_a_guardar = f"Experimento #{experimento}: Config={configuracion}"
    if experimento % 100 == 0:
        df_resultados = pd.DataFrame(resultados)
        df_resultados.to_csv("resultados_exp_22_11_V2.csv", index=False)
    resultados.append(configuracion)
    
    guardar_log(texto_a_guardar, archivo="bitacora_experimentos_exp_22_11_V2.txt")
    
    if best_f1 > 0.82:
        nombre_archivo = f'../Resultados_Leo_V3/experimentos_V3_{experimento}.csv'
        guardar_resultados(y_pred, nombre_archivo)
    
    experimento += 1



Ep 1 | Val Loss: 0.4505 | F1 Macro: 0.7630 (Humor F1: 0.6761)
¡Nuevo Récord! Modelo guardado.
Ep 2 | Val Loss: 0.4626 | F1 Macro: 0.7143 (Humor F1: 0.5890)
Ep 3 | Val Loss: 0.4349 | F1 Macro: 0.7787 (Humor F1: 0.7166)
¡Nuevo Récord! Modelo guardado.
Ep 4 | Val Loss: 0.4376 | F1 Macro: 0.7698 (Humor F1: 0.6978)
Ep 5 | Val Loss: 0.4419 | F1 Macro: 0.7576 (Humor F1: 0.6660)
Ep 6 | Val Loss: 0.4994 | F1 Macro: 0.7687 (Humor F1: 0.7159)
Ep 7 | Val Loss: 0.4920 | F1 Macro: 0.7795 (Humor F1: 0.7251)
¡Nuevo Récord! Modelo guardado.
Ep 8 | Val Loss: 0.5177 | F1 Macro: 0.7615 (Humor F1: 0.7204)
Ep 9 | Val Loss: 0.4804 | F1 Macro: 0.7783 (Humor F1: 0.7164)
Ep 10 | Val Loss: 0.5013 | F1 Macro: 0.7827 (Humor F1: 0.7214)
¡Nuevo Récord! Modelo guardado.
Ep 11 | Val Loss: 0.5531 | F1 Macro: 0.7844 (Humor F1: 0.7242)
¡Nuevo Récord! Modelo guardado.
Ep 12 | Val Loss: 0.5712 | F1 Macro: 0.7764 (Humor F1: 0.7245)
Ep 13 | Val Loss: 0.5733 | F1 Macro: 0.7809 (Humor F1: 0.7173)
Ep 14 | Val Loss: 0.6218 | F1 

KeyboardInterrupt: 

In [29]:
configuracion = {
        "capas": [1024, 1024, 1024],
        "batch_size": 48,
        "learning_rate": 0.1,
        "patience": 40,
        "epochs": 256,
        "factor_balance": 0.75
    }

y_pred, _, _ = ejecutar_experimento(configuracion["capas"], configuracion["batch_size"], configuracion["learning_rate"],
                                                        configuracion["epochs"], patience=configuracion["patience"], factor_balance=configuracion["factor_balance"])

nombre_archivo = '../Resultados_Leo/experimentos_176.csv'

C:\Users\leopa\AppData\Local\Temp\ipykernel_11464\3315706254.py:94: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))



--- Reporte de Clasificación ---
              precision    recall  f1-score   support

    No Humor     0.8650    0.8755    0.8702       988
       Humor     0.7804    0.7640    0.7721       572

    accuracy                         0.8346      1560
   macro avg     0.8227    0.8197    0.8212      1560
weighted avg     0.8340    0.8346    0.8342      1560

Cargando dataset de prueba...
Procesando vectores...
Dimensiones de Test listas: (5600, 900)
Predicciones generadas:
[0 1 0 ... 1 0 1]

Conteo de clases predichas:
{np.int64(0): np.int64(3513), np.int64(1): np.int64(2087)}

Experimento finalizado. Mejor F1 Macro: 0.8212


In [30]:
guardar_resultados(y_pred,nombre_archivo)

 [0 1 0 ... 1 0 1] guardado exitosamente!
